In [3]:
import db
import networkx as nx
import pandas as pd
import psycopg2

In [4]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import os

os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/"

findspark.init("./spark-3.2.0-bin-hadoop3.2")

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

21/11/16 12:52:08 WARN Utils: Your hostname, 127.0.0.1 resolves to a loopback address: 127.0.0.1; using 10.105.216.178 instead (on interface en0)
21/11/16 12:52:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/16 12:52:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/16 12:52:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
tables = db.tables()
set(tables['table_name'])

{'activity_grid_activitycard',
 'activity_grid_activitypaircard',
 'activity_log_activitylog',
 'analytics_attachmenttracking',
 'analytics_event',
 'analytics_searchtracking',
 'app_config_appconfig',
 'app_config_visualtokencolor',
 'auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'authtoken_token',
 'cms_slugpage',
 'data_allegation',
 'data_allegation_areas',
 'data_allegation_line_areas',
 'data_allegationcategory',
 'data_area',
 'data_attachmentfile',
 'data_attachmentfile_tags',
 'data_attachmentnarrative',
 'data_attachmentocr',
 'data_attachmentrequest',
 'data_award',
 'data_complainant',
 'data_crew',
 'data_investigator',
 'data_investigatorallegation',
 'data_involvement',
 'data_linearea',
 'data_officer',
 'data_officeralias',
 'data_officerallegation',
 'data_officerassignmentattendance',
 'data_officerbadgenumber',
 'data_officercrew',
 'data_officerhistory',
 'data_officerrank',
 'data_off

In [6]:
racepop = db.download("data_racepopulation")
racepop.shape

(517, 6)

In [7]:
salary = db.download("data_salary")
salary.shape

(209219, 14)

In [8]:
area = db.download("data_area")
area.shape

(1551, 12)

In [16]:
allegation_line_areas = db.download("data_allegation_line_areas")
allegation_line_areas.shape

(4307, 3)

In [11]:
allegation_areas = db.download("data_allegation_areas")
allegation_areas.shape

(928000, 3)

In [15]:
allegation_areas

,id,allegation_id,area_id
0,266400,C146063,338
1,266401,C146063,1528
2,266402,C146123,1528
3,266403,C146123,393
4,266404,C146123,456
...,...,...,...
927995,266395,C146030,435
927996,266396,C146030,380
927997,266397,C146063,42
927998,266398,C146063,431


In [12]:
officerallegation = db.download("data_officerallegation")
officerallegation.shape

(249788, 15)

In [16]:
officerallegation

,id,start_date,end_date,officer_age,recc_finding,recc_outcome,final_finding,final_outcome,final_outcome_class,allegation_category_id,officer_id,disciplined,created_at,updated_at,allegation_id
0,510,1983-08-13,1986-12-03,None,,Unknown,SU,Separation,,104.0,13617,True,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,C132885
1,1582,1986-03-15,1989-06-14,None,,Unknown,SU,Separation,,98.0,23618,True,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,C149900
2,1738,1986-06-24,1988-10-18,None,,Unknown,SU,Separation,,53.0,3049,True,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,C151502
3,1797,1986-08-04,1987-03-09,None,,Unknown,SU,Reprimand,,185.0,5124,True,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,C152222
4,2393,1987-07-30,1990-04-11,None,,Unknown,SU,Separation,,197.0,5383,True,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,C157296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249783,244412,2016-06-27,2016-12-01,None,,Unknown,NS,Unknown,,111.0,12001,None,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,1081167
249784,244413,2016-06-28,2017-02-06,None,,Unknown,NA,Unknown,,333.0,10321,None,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,1081169
249785,244414,2016-06-28,2017-05-17,None,,Unknown,NA,Unknown,,113.0,26802,None,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,1081170
249786,244415,2016-06-28,2017-05-17,None,,Unknown,NA,Unknown,,113.0,27962,None,2019-01-08 22:41:59.265501-06:00,2019-01-08 22:42:02.108401-06:00,1081170


In [13]:
officer = db.download("data_officer")
officer.shape

(35545, 33)

In [18]:
officer

,id,gender,race,appointed_date,rank,active,birth_year,first_name,last_name,tags,...,current_salary,discipline_count,honorable_mention_count,last_unit_id,major_award_count,trr_count,unsustained_count,has_unique_name,created_at,updated_at
0,32312,M,White,1998-11-30,Detective,Unknown,1973.0,Randall,Ryan,[],...,97440.0,0.0,8.0,151.0,0.0,4.0,2.0,False,2019-01-08 22:41:58.087320-06:00,2019-01-08 22:41:58.547020-06:00
1,32358,M,White,1995-06-05,Detective,Unknown,1968.0,Kevin,Stoll,[],...,100980.0,0.0,55.0,151.0,0.0,5.0,11.0,True,2019-01-08 22:41:58.087320-06:00,2019-01-08 22:41:58.547020-06:00
2,32480,M,White,2000-07-05,,Unknown,1967.0,David,Lavin,[],...,NaN,0.0,1.0,NaN,0.0,0.0,0.0,False,2019-01-08 22:41:58.087320-06:00,2019-01-08 22:41:58.547020-06:00
3,32491,M,White,1998-10-01,,Unknown,1942.0,Kevin,Rake,[],...,NaN,0.0,4.0,NaN,0.0,0.0,0.0,False,2019-01-08 22:41:58.087320-06:00,2019-01-08 22:41:58.547020-06:00
4,2680,M,White,1956-10-12,,No,1932.0,Richard,Brandt,[],...,NaN,0.0,0.0,83.0,0.0,0.0,0.0,True,2019-01-08 22:41:58.087320-06:00,2019-01-08 22:41:58.547020-06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35540,32098,M,White,1992-06-29,Deputy Chief,Unknown,1970.0,Christoph,Kennedy,[],...,162684.0,0.0,6.0,265.0,0.0,2.0,5.0,True,2019-01-08 22:41:58.087320-06:00,2021-01-05 19:47:35.337732-06:00
35541,25962,M,White,1997-09-02,Police Officer,No,1966.0,Joseph,Seinitz,[skullcap],...,68262.0,2.0,2.0,6.0,0.0,0.0,21.0,True,2019-01-08 22:41:58.087320-06:00,2021-01-05 19:47:35.280025-06:00
35542,25503,M,White,1995-07-10,Police Officer,Yes,1969.0,Christ,Savickas,[skullcap],...,93354.0,0.0,50.0,225.0,0.0,7.0,17.0,False,2019-01-08 22:41:58.087320-06:00,2021-01-05 19:47:35.337732-06:00
35543,25732,M,White,1997-08-04,Sergeant of Police,Yes,1969.0,Andrew,Schoeff,"[Shoeff, Sheff, Chef, Shef, Shoff, skullcap]",...,107988.0,1.0,6.0,215.0,1.0,1.0,14.0,True,2019-01-08 22:41:58.087320-06:00,2021-01-05 19:47:34.711074-06:00


In [14]:
allegation = db.download("data_allegation")
allegation.shape

(216052, 20)

In [20]:
trr = db.download("trr_trr")
trr.shape

(67019, 42)

In [51]:
temp = trr['event_id'].value_counts() > 1
coaccused = set([i for i in temp.index if temp[i] == True])
len(coaccused)

17752

In [52]:
trr[trr['event_id'].isin(coaccused)]

,id,beat,block,direction,street,location,trr_datetime,indoor_or_outdoor,lighting_condition,weather_condition,...,subject_gender,subject_race,officer_id,officer_unit_id,officer_unit_detail_id,point,created_at,updated_at,crid,event_id
0,3810,423,26XX,East,93Rd St,Street,2004-09-14 10:46:00-05:00,Outdoor,DAYLIGHT,CLEAR,...,M,BLACK,4270.0,5,NaN,0101000020E610000090A56A16C3E355C0CBBE2B82FFDC...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,None,425810831
1,3811,423,26XX,East,93Rd St,Street,2004-09-14 10:46:00-05:00,Outdoor,DAYLIGHT,CLEAR,...,M,BLACK,9596.0,30,5.0,0101000020E610000090A56A16C3E355C0CBBE2B82FFDC...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,None,425810831
2,3815,734,12XX,West,72Nd Pl,Residence,2004-09-14 12:20:00-05:00,Indoor,DAYLIGHT,CLEAR,...,F,BLACK,32369.0,30,8.0,0101000020E610000025276E70FDE955C0E15F048D99E1...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,None,425813116
3,3817,734,12XX,West,72Nd Pl,Residence,2004-09-14 12:20:00-05:00,Indoor,DAYLIGHT,CLEAR,...,F,BLACK,28476.0,8,NaN,0101000020E610000025276E70FDE955C0E15F048D99E1...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,None,425813116
4,3821,324,72XX,South,Woodlawn Ave,Sidewalk,2004-09-14 13:14:00-05:00,Outdoor,DAYLIGHT,CLEAR,...,M,BLACK,3716.0,4,NaN,0101000020E6100000E61F7D9326E655C0D2B30F689FE1...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,300821,425813861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67013,3800,1322,18XX,West,Augusta Blvd,Other,2004-09-13 16:19:00-05:00,Outdoor,NIGHT,CLEAR,...,M,WHITE,29502.0,14,NaN,0101000020E6100000E0BE695526EB55C09C3061342BF3...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,None,425718291
67015,3802,331,68XX,South,Chappel Ave,Street,2004-09-13 19:47:00-05:00,Outdoor,GOOD ARTIFICIAL,CLEAR,...,M,BLACK,9697.0,4,NaN,0101000020E61000002EB94038D5E455C0DEE2868494E2...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,300697,425800613
67016,3803,331,68XX,South,Chappel Ave,Street,2004-09-13 19:47:00-05:00,Outdoor,GOOD ARTIFICIAL,CLEAR,...,M,BLACK,32208.0,4,NaN,0101000020E61000002EB94038D5E455C0DEE2868494E2...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,300697,425800613
67017,3804,434,117XX,South,Torrence Ave,Other,2004-09-13 23:50:00-05:00,None,None,None,...,M,BLACK,22235.0,120,NaN,0101000020E61000007DD8C177D1E355C008043A9336D7...,2019-01-08 22:43:07.860473-06:00,2019-01-08 22:43:11.294093-06:00,None,425802634


In [29]:
# find officers that engaged in use of force with many other officers